In [9]:
import openpyxl
import math
import random
import matplotlib.pyplot as plt
import imageio
import numpy as np

In [11]:
class KNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = []
        self.y_train = []

    def fit(self, X_train, y_train):
        """Stores the training data."""
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test, save_path="knn_animation.gif"):
        """Predict labels and create animation."""
        frames = []
        predictions = []
        
        for i, x in enumerate(X_test):
            pred, frame = self._predict_and_plot(x, i)
            predictions.append(pred)
            frames.append(frame)
        
        # Save GIF animation
        imageio.mimsave(save_path, frames, duration=0.5)
        return predictions

    def _predict_and_plot(self, x, test_idx):
        """Classify a single test sample and generate a frame for animation."""
        distances = []
        for i in range(len(self.X_train)):
            dist = self._euclidean_distance(x, self.X_train[i])
            distances.append((dist, self.y_train[i], self.X_train[i]))

        # Sort and pick k nearest neighbors
        distances.sort(key=lambda d: d[0])
        k_nearest = distances[:self.k]

        # Determine majority class
        k_labels = [label for _, label, _ in k_nearest]
        predicted_class = max(set(k_labels), key=k_labels.count)

        # Plot training data
        fig, ax = plt.subplots(figsize=(8, 6))
        for i in range(len(self.X_train)):
            color = 'blue' if self.y_train[i] == 0 else 'red'
            ax.scatter(self.X_train[i][0], self.X_train[i][1], color=color, alpha=0.6)

        # Plot test point
        ax.scatter(x[0], x[1], color="black", s=100, label="Test Point")

        # Draw lines to k nearest neighbors
        for _, label, neighbor in k_nearest:
            color = 'blue' if label == 0 else 'red'
            ax.plot([x[0], neighbor[0]], [x[1], neighbor[1]], color=color, linestyle="--")

        # Annotate the frame
        ax.set_title(f"KNN Classification: Test Point {test_idx+1}\nPredicted Class: {predicted_class}")
        ax.set_xlabel("Feature 1")
        ax.set_ylabel("Feature 2")
        ax.legend()
        
        # Save frame
        fig.canvas.draw()
        image = np.array(fig.canvas.renderer.buffer_rgba())
        plt.close(fig)
        return predicted_class, image

    def _euclidean_distance(self, point1, point2):
        """Calculate Euclidean distance between two points."""
        return math.sqrt(sum((point1[i] - point2[i]) ** 2 for i in range(len(point1))))

In [13]:
# Load dataset manually from Excel
file_path = "DataSetNew-GPS.xlsx"  # Change this to your actual file
wb = openpyxl.load_workbook(file_path)
sheet = wb.active

# Read data manually
data = []
for row in sheet.iter_rows(values_only=True):
    data.append(row)

# Remove header if it exists
data = data[1:]  

# Randomly select 1000 rows from 240000
random.shuffle(data)
data = data[:200]

# Extract features and labels
X = [list(row[:2]) for row in data]  # First two columns as features
y = [row[2] for row in data]         # Third column as label (0 or 1)

# Split data manually (80% train, 20% test)
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [14]:
# Initialize and train KNN model
knn = KNN(k=5)
knn.fit(X_train, y_train)

# Make predictions and generate animation
y_pred = knn.predict(X_test, save_path="knn_classification.gif")

# Calculate accuracy manually
correct = sum(1 for i in range(len(y_test)) if y_test[i] == y_pred[i])
accuracy = (correct / len(y_test)) * 100

print(f"\nKNN Classification Accuracy: {accuracy:.2f}%")
print("KNN Animation GIF Saved Successfully!")


KNN Classification Accuracy: 99.00%
KNN Animation GIF Saved Successfully!


In [19]:
def frange(start, stop, step):
    while start < stop:
        yield round(start, 10)  # Avoid floating-point precision issues
        start += step

# Create a mesh grid for decision boundary visualization
x_min, x_max = min(p[0] for p in X), max(p[0] for p in X)
y_min, y_max = min(p[1] for p in X), max(p[1] for p in X)

# Generate a grid of points
step = 0.1  # Adjust resolution
xx, yy = [], []
x_range = list(frange(x_min - 1, x_max + 1, step))
y_range = list(frange(y_min - 1, y_max + 1, step))

for x in x_range:
    for y in y_range:
        xx.append(x)
        yy.append(y)

grid_points = list(zip(xx, yy))

# Predict class for each grid point
grid_predictions = knn.predict(grid_points)

# Plot decision boundary
plt.figure(figsize=(8, 6))
for i in range(len(grid_points)):
    color = "lightblue" if grid_predictions[i] == 0 else "lightcoral"
    plt.scatter(grid_points[i][0], grid_points[i][1], color=color, s=2, alpha=0.5)

# Plot training data
for i in range(len(X_train)):
    color = "blue" if y_train[i] == 0 else "red"
    plt.scatter(X_train[i][0], X_train[i][1], color=color, edgecolor="k", s=30)

plt.title("KNN Decision Boundary")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()



KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x0000004AB8A08680> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



Error in callback <function flush_figures at 0x0000004ABAE28E00> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [21]:
import imageio

def slow_down_gif(input_gif, output_gif, slow_factor=5):
    """Slows down a GIF by increasing frame duration."""
    # Load GIF frames
    gif = imageio.mimread(input_gif)
    
    # Get the original duration (default is ~0.1s if not specified)
    original_duration = 0.1  
    
    # New duration (5x slower)
    new_duration = original_duration * slow_factor

    # Save new GIF with slower frame rate
    imageio.mimsave(output_gif, gif, duration=new_duration)

# Example usage
slow_down_gif("knn_classification.gif", "knn_classification2.gif", slow_factor=5)

print("GIF slowed down successfully!")

ValueError: all input arrays must have the same shape